In [1]:
from hybridbrep import GeneralConvEncDec, BRepFaceAutoencoder, HybridPartDataset
import json
from zipfile import ZipFile
import torch
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from train_latent_space import BRepFaceAutoencoder as OldAutoencoder, BRepDS as OldDS
import altair as alt
import pandas as pd

In [2]:
old_model = BRepFaceAutoencoder(64, 1024, 4, True)
new_model = GeneralConvEncDec(64, 1024, 4)

old_ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/old_with_loops/version_0/checkpoints/epoch=54-val_loss=0.002403.ckpt'
new_ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/new_with_edges/version_1/checkpoints/epoch=183-val_loss=0.002646.ckpt'

old_ckpt = torch.load(old_ckpt_path)
new_ckpt = torch.load(new_ckpt_path)

old_model.load_state_dict(old_ckpt['state_dict'])
new_model.load_state_dict(new_ckpt['state_dict'])

<All keys matched successfully>

In [3]:
with ZipFile('../../datasets/fusion360seg_hpart_fixed.zip','r') as zf:
    names = [n for n in zf.namelist() if n.endswith('.stp')]
    data = []
    for n in tqdm(names):
        with zf.open(n, 'r') as f:
            data.append((n, torch.load(f)))

100%|██████████| 27450/27450 [01:12<00:00, 376.07it/s]


In [6]:
data[0][1].face_to_face

tensor([[ 7,  5,  6,  1,  7,  6,  1,  7,  6,  2,  7,  6,  4,  4,  6,  5,  5,  6],
        [ 0,  0,  0,  0,  1,  1,  2,  2,  2,  3,  3,  3,  3,  7,  4,  4,  7,  5],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [7]:
with open('../../datasets/fusion360seg.json','r') as f:
    index = json.load(f)

In [12]:
labels = {}

for i in range(len(index['train'])):
    k = index['train'][i]
    n = index['template'].format(*k)
    l = torch.tensor(index['train_labels'][i]).long()
    labels[n] = l

for i in range(len(index['test'])):
    k = index['test'][i]
    n = index['template'].format(*k)
    l = torch.tensor(index['test_labels'][i]).long()
    labels[n] = l

In [14]:
old_precoded = {}
new_precoded = {}
for n, hdata in tqdm(data):
    y = labels[n]
    x_old = old_model.encode(hdata).float()
    x_new = new_model.encode_faces(hdata).float()
    edge_index = hdata.face_to_face[:2].long()
    old_precoded[n] = {
        'x':x_old,
        'y':y,
        'edge_index':edge_index
    }
    new_precoded[n] = {
        'x':x_new,
        'y':y,
        'edge_index':edge_index
    }

100%|██████████| 27450/27450 [00:48<00:00, 564.51it/s]


In [17]:
torch.save(old_precoded,'../../datasets/fusion360seg_precoded_old_model_new_data.pt')
torch.save(new_precoded,'../../datasets/fusion360seg_precoded_new_model_new_data.pt')